## Use agents.json with Google Sheets API

In [14]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
%pip install openai google-auth-oauthlib google-auth-httplib2 google-auth

### Get Test Auth Credentials
1. Go to https://developers.google.com/oauthplayground/
2. Find "Sheets v4" API
3. Select all scopes
4. Click on "Exchange authorization code for tokens"
5. Add values below

In [23]:
auth_data = {
  "access_token": "",
  "scope": "https://www.googleapis.com/auth/spreadsheets",
  "token_type": "Bearer",
  "expires_in": 3600,
  "refresh_token": ""
}

In [24]:
import json
from typing import Optional
from google.oauth2.credentials import Credentials

def load_auth_from_json(auth_json: str, auth_path: Optional[str] = None) -> Credentials:
    """
    Load Google OAuth2 credentials directly from a JSON file.
    
    Args:
        auth_path: Path to the auth JSON file containing tokens
        
    Returns:
        Google OAuth2 Credentials object
    """
    
    if auth_path:
        with open(auth_path, 'r') as f:
            token_data = json.load(f)
    else:
        token_data = json.loads(auth_json)
    
    return Credentials(
        token=token_data['access_token'],
        refresh_token=token_data['refresh_token'],
        token_uri='https://oauth2.googleapis.com/token',
        client_id='',  # These are not needed for sandbox
        client_secret='',
        scopes=token_data['scope'].split(' ')
    )

### Create a Google Sheets Agent
This class encapsulates an agent with the following functions.
1. Setup Google Auth - This function sets up the Google Sheets API credentials.
2. Load Agents.json - This function loads the agents.json file.
3. Execute Query - This function executes a natural language query against Google Sheets.





In [ ]:
import yaml
from typing import Dict, Any, List

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from openai import OpenAI

from agentsjson.core.models import Flow
from agentsjson.core.models.bundle import Bundle
from agentsjson.core.models.auth import AuthType, OAuth2AuthConfig
import agentsjson.core as core
from agentsjson.core.executor import execute_flows
from agentsjson.core import ToolFormat

class GoogleSheetsAgent:
    def __init__(self, auth_json_path: str, openai_api_key: str):
        """
        Initialize the Google Sheets Agent.
        
        Args:
            auth_json_path: Path to the auth JSON file containing tokens
            openai_api_key: OpenAI API key
        """
        self.auth_json_path = auth_json_path
        self.openai_api_key = openai_api_key
        self.google_creds = None
        self.bundle = None
        self.flows = None
        
        # Initialize OpenAI client
        self.openai_client = OpenAI(api_key=openai_api_key)

    def setup_google_auth(self) -> None:
        """Set up Google OAuth2 authentication using the auth JSON file."""
        if not os.path.exists(self.auth_json_path):
            raise FileNotFoundError(f"Auth JSON file not found at {self.auth_json_path}")
        
        self.google_creds = load_auth_from_json(self.auth_json_path)
        print("Successfully loaded Google Sheets API credentials")

    def load_agents_json(self, agents_json_path: str) -> None:
        """Load the agents.json file."""
        try:
            # First load the agents.json content
            with open(agents_json_path, 'r') as f:
                agents_json_content = json.load(f)
            
            # Get the OpenAPI spec path from the sources
            openapi_path = os.path.join(os.path.dirname(agents_json_path), 'openapi.yaml')
            
            # Load the OpenAPI spec
            with open(openapi_path, 'r') as f:
                openapi_content = yaml.safe_load(f)
            
            # Create the bundle structure
            bundle_data = {
                "agentsJson": agents_json_content,
                "openapi": openapi_content,
                "operations": {}  # This will be populated by the Bundle model
            }
            
            self.bundle = Bundle.model_validate(bundle_data)
            self.flows = self.bundle.agentsJson.flows
            print("Successfully loaded agents.json and OpenAPI spec")
        except FileNotFoundError as e:
            raise Exception(f"File not found: {str(e)}")
        except json.JSONDecodeError as e:
            raise Exception(f"Invalid JSON in agents.json: {str(e)}")
        except Exception as e:
            raise Exception(f"Error loading agents.json: {str(e)}")

    def execute_query(self, query: str, flow_hint: Optional[List[str]] = None) -> Dict[str, Any]:
        """Execute a natural language query against Google Sheets."""
        if not self.google_creds or not self.bundle or not self.flows:
            raise Exception("Please call setup_google_auth() and load_agents_json() first")

        # Format flows for the prompt
        flows = self.flows        
        if flow_hint:
            flows = [flow for flow in self.flows if flow.id in flow_hint]
                    
        flows_context = core.flows_prompt(flows)
        
        # Create system prompt
        system_prompt = f"""You are an AI assistant that helps users interact with the Google Sheets API.
You have access to the following API flows:

{flows_context}

Analyze the user's request and use the appropriate API flows to accomplish the task.
You must give your arguments for the tool call as Structured Outputs JSON with keys `parameters` and `requestBody`.
When creating or updating spreadsheets, make sure to format the data in a clear and organized way.

For the spreadsheets_create flow, provide the requestBody in this format:
"""


        tools = core.flows_tools(flows, format=ToolFormat.OPENAI)
        print("TOOLS", tools)

        # Get completion from OpenAI
        response = self.openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query}
            ],
            tools=core.flows_tools(flows, format=ToolFormat.OPENAI),
            temperature=0
        )
        
        # Configure auth
        auth = OAuth2AuthConfig(
            type=AuthType.OAUTH2,
            credentials=self.google_creds,
            token=self.google_creds.token,
            scopes=set(self.google_creds.scopes),
            refresh_token=self.google_creds.refresh_token,
        )
        
        # Execute the flows
        result = execute_flows(
            response,
            format=ToolFormat.OPENAI,
            bundle=self.bundle,
            flows=flows,
            auth=auth
        )
        
        return result

In [19]:
current_dir = os.path.dirname("/Users/kaushik/Documents/OceanWater/agents-json/examples/sheets.ipynb")
auth_json_path = os.path.join(current_dir, "sheetsauth.json")
agents_json_path = os.path.join(current_dir, "..", "agents_json", "googlesheets", "agents.json")

In [ ]:

openai_api_key = os.getenv("OPENAI_API_KEY")
agent = GoogleSheetsAgent(auth_json_path, openai_api_key)
agent.setup_google_auth()
agent.load_agents_json(agents_json_path)


In [ ]:
query = "Create a new spreadsheet titled 'Customer Relationship Management' and add the columns 'Name', 'Email', 'Phone', 'Address', 'City', 'State', 'Zip', 'Country'. Then add 10 rows of sample data to it."
# flow_hint = [] - Optional to help the LLM choose the right flow

result = agent.execute_query(query)
result